In [1]:
%run "KiteConnect_Library.ipynb"   

warnings.filterwarnings('ignore')

mode = "Live" # Backtesting/Live/Algo
offline = False
if mode =="Live":
    offline = False 

### Filter Data

In [77]:
oi_url = "https://www.nseindia.com/products/content/equities/equities/oi_spurts.htm"

In [87]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
import re
import html5lib
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def download_from_nse(symbol):
    driver = webdriver.Chrome("./automation/chromedriver")
    driver.get("https://www.nseindia.com/products/content/equities/equities/eq_security.htm")
    #assert "Python" in driver.title
    elem = driver.find_element_by_name("symbol")
    elem.clear()
    elem.send_keys(symbol)

    dateRange = Select(driver.find_element_by_name("dateRange"))
    dateRange.select_by_value("week")


    dateRange = Select(driver.find_element_by_name("series"))
    dateRange.select_by_value("EQ")

    #submitBtn = driver.find_element_by_id("submitMe")
    #elem.submit()

    driver.execute_script("submitData()")

    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "download-data-link"))
        )
        oi_html= driver.find_element_by_id('historicalData').get_attribute('innerHTML')

        #f_states=   pd.read_html('https://simple.wikipedia.org/wiki/List_of_U.S._states') 
        stock_data = pd.read_html(oi_html, header=[0])[0]

    finally:
        driver.quit()
        
    return stock_data

In [89]:
import pandas as pd
import html5lib
driver = webdriver.Chrome("./automation/chromedriver")
driver.get(oi_url)

try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "tab7Content"))
    )
    oi_html= driver.find_element_by_id('tab7Content').get_attribute('innerHTML')

    #f_states=   pd.read_html('https://simple.wikipedia.org/wiki/List_of_U.S._states') 
    data1 = pd.read_html(oi_html, header=[0])[0][1::]

finally:
    driver.quit()
    

In [179]:
temp_df = pd.DataFrame()

In [180]:
comp = lambda a,b: 1 if a>= b else 0

for index, row in data1.iterrows():
    symbol = row['Symbol']
    if re.findall('NIFTY',symbol):
        continue
    
    if re.findall('KPIT',symbol):
        symbol = 'BSOFT'
        continue
        
    oi_change = row['% Chg in OI']
    stock_data = download_from_nse(symbol)
    stock_data2 = stock_data['% Dly Qt toTraded Qty']
    
    mysum = comp(stock_data2.iloc[-1],stock_data2.iloc[-2])+comp(stock_data2.iloc[-2],stock_data2.iloc[-3])+comp(stock_data2.iloc[-3],stock_data2.iloc[-4])

    temp_data = [{'symbol':stock_data.iloc[-1]['Symbol'], 'oi_%_change':oi_change, 
                 'Today -3':stock_data2.iloc[-4], 'Today -2':stock_data2.iloc[-3], 'Today -1':stock_data2.iloc[-2],'sum':mysum,
                 'Today':stock_data2.iloc[-1], 'Prev Close':stock_data.iloc[-1]['Prev Close'], 'open':stock_data.iloc[-1]['Open Price'], 'close':stock_data.iloc[-1]['Close Price'], 'Turnover':stock_data.iloc[-1]['Turnover']}]
    #print(temp_data)

    df = pd.DataFrame.from_records(temp_data, index='symbol')
    #print(df)
    temp_df = temp_df.append(df)
    #break

In [183]:
temp_df.to_csv('data/nse_filter_data.csv')

In [186]:
raw_data = temp_df.sort_values('Turnover')

In [198]:
check1 = temp_df['close']>temp_df['open']
check2 = temp_df['close']>temp_df['Prev Close']
chk = temp_df['Today -2']>temp_df['Today -3']
chk1 = temp_df['Today -1']>temp_df['Today -2']
chk2 = temp_df['Today']>temp_df['Today -1']


temp_df['oi_%_change']

long = (temp_df['Today']>60) & (temp_df['sum']>=2)& (chk2 == True)&((check1==True) | (check2==True))
short = (temp_df['Today']<=20) & (temp_df['sum']<=1)& (chk2 == False)&((check1==False) | (check2==False))

In [200]:
temp_df[long]

,Prev Close,Today,Today -1,Today -2,Today -3,Turnover,close,oi_%_change,open,sum
symbol,,,,,,,,,,
MARICO,364.0,60.12,41.56,60.19,30.6,651614369.5,366.1,-1.56,362.25,2


In [204]:
temp_df[short].sort_values('Turnover', ascending=False)

,Prev Close,Today,Today -1,Today -2,Today -3,Turnover,close,oi_%_change,open,sum
symbol,,,,,,,,,,
PCJEWELLER,145.95,6.56,8.45,6.71,8.98,1.800326e+10,145.15,-11.82,149.00,1
INDIGO,1581.90,17.71,22.51,14.44,26.56,1.335256e+10,1554.40,16.10,1597.00,1
RBLBANK,680.30,14.27,24.33,26.89,27.89,2.315978e+09,675.45,17.85,684.20,0
ASHOKLEY,96.35,16.06,20.80,23.93,22.73,1.889255e+09,95.75,0.29,97.00,1
DCBBANK,202.25,9.03,16.60,28.23,25.56,1.759174e+09,202.30,19.00,203.45,1
PVR,1716.85,17.28,28.09,36.78,30.80,1.064478e+09,1710.05,1.63,1719.70,1
ULTRACEMCO,4255.05,18.22,39.11,49.00,33.08,8.972604e+08,4259.95,-1.66,4279.70,1
PEL,2672.95,14.83,26.73,35.38,32.72,6.302912e+08,2649.85,5.66,2681.60,1
BHEL,76.70,12.12,19.52,28.88,42.89,5.482854e+08,76.05,-0.27,78.00,0


In [2]:
portfolio = pd.DataFrame(downloadlist.tolist())
portfolio = pd.DataFrame(['TATAMOTORS','WIPRO','HDFCBANK'])

portfolioToken = portfolio[0].apply(EQTOKEN).tolist()

### Trading Strategy

In [225]:
#pivotPoint(raw_data, pos=1, plot=False)
#raw_data = calculateStats(raw_data)
#raw_data = emasma(raw_data)
#raw_data = adx(raw_data)
#raw_data = detectCDPattern(raw_data,strPlot="hammer", plot=True)
#raw_data = stoch(raw_data)

# ====== Tradescript Wrapper =======
# Variables
OPEN = 0
CLOSE = 0
HIGH = 0
LOW = 0
UP = 0
DOWN = 1
iRSI = 0
STATE = "SELL"
Fast_K = 0
Slow_K = 0
Slow_D = 0

# Methods
REF = lambda df, i: df.shift(i)
TREND_UP = lambda a, days: ROC(a,days) >= 0.01
TREND_DOWN = lambda a, days: ROC(a,days) <= -0.01
#TREND_UP = lambda high, low: ((REF(high,1) >= REF(high,2)) & (REF(low,1) >= REF(low,2)))
#TREND_DOWN =  lambda high, low: ((REF(high,1) <= REF(high,2)) & (REF(low,1) <= REF(low,2)))

CROSSOVER = lambda a, b: (REF(a,1)<=REF(b,1)) & (a > b)
profit = 0
stoploss = 0
squareoff = 0


from talib import MIN, MAX
class algoTrade:
    
    def __init__(s, price):
        #logging.debug(price.tail(10))
        global OPEN, CLOSE, HIGH, LOW, iRSI, sma20, sma50, Fast_K, Slow_K, Slow_D
        
        OPEN = (price.open.shift(1) + price.close.shift(1))/2
        HIGH = price[['high','open','close']].max(axis = 1, skipna = True)
        LOW = price[['low','open','close']].min(axis = 1, skipna = True)
        CLOSE = (price.open+price.high+price.low+price.close)/4
        
        
        k_period = 40
        d_period = 10
        kma = 3
        dma = 3
        
        mlow = MIN(LOW, timeperiod=k_period)
        mhigh = MAX(HIGH, timeperiod=k_period)
        
        Fast_K = 100 * SMA( ( CLOSE[-1] - mlow)/( mhigh - mlow), k_period)
        Slow_K = SMA(Fast_K, kma)
        Slow_D = SMA(Slow_K, dma)
        iRSI = RSI(CLOSE, timeperiod=20)
    
    def long_pa(self):
        return 0
    def short_pa(self):
        return 0
 
    
    def tradeDecision(self):
        buy  = self.long_pa() # |self.long_macd_crossover() # | self.long_bb() #| self.long_ichimoku() #| self.long_bull_engulf_ema() #| self.long_ichimoku()
        
        sell = self.short_pa() # |self.short_macd_crossover() # | self.short_bb() #| self.short_bear_engulf() #| self.short_hanging_man() #|  self.short_ichimoku()
        
        buy['low'] = LOW
        sell['high'] = HIGH
        buy['close'] = sell['close'] = CLOSE
        buy['sl'] = CLOSE - SD
        sell['sl'] = CLOSE + SD
        buy['so'] = CLOSE + 4 * SD
        sell['so'] = CLOSE - 4 * SD

        buy = buy[buy['buy']]
        sell = sell[sell['sell']]
        return (buy,sell)
        
    def tradeBot(self, symbol):
        global STATE
        buy, sell = self.tradeDecision()
        logger.debug(symbol+","+STATE)
        timeNow = dt.datetime.now().replace(second=0, microsecond=0)
        delT1 = timeNow - datetime.timedelta(minutes = 1)
        delT2 = timeNow - datetime.timedelta(minutes = 2)
        
        price = CLOSE[-1]
        trigger = CLOSE[-1]
        #stoploss = SD[-1]
        stoploss = 0
        squareoff = 3 * SD[-1] 
        
        logger.debug(buy)
        logger.debug(sell)
        
        if buy.empty | sell.empty:
            return
        
        if (buy.empty != True) & ((delT1 == buy.index[-1]) | (delT2 == buy.index[-1])) & (STATE != "SELL"):
            #buy_bo(symbol, price, trigger, stoploss, squareoff, quantity=1)
            buy_sl(symbol, price, trigger, quantity=1)
            STATE = "SELL"
        elif (sell.empty != True) & ((delT1 == sell.index[-1]) | (delT2 == sell.index[-1])) & (STATE != "BUY"):
            STATE = "BUY"
            #sell_bo(symbol, price, trigger, stoploss, squareoff, quantity=1)
            sell_sl(symbol, price, trigger, quantity=1)
            
    
            
        
papertrade = False
if True:
    global fig
    noofcandles = 60
    x = "MARICO"
    toDate = dt.datetime(2019,4,18,3,30)
    temp_data = downloadData(x, fromDate = toDate - dt.timedelta(days=2), toDate=toDate, freq="minute")
    
    #temp_data = resample2(temp_data.loc[symbol],"1min").dropna()[-20*noofcandles:-1]
    #temp_data = candlestick(temp_data,1,True)
    
    #temp_data = rsi(temp_data,4, True)
    
    a = algoTrade(temp_data)
    #buy, sell = a.tradeDecision()

In [4]:
if mode == "Live":
    kws.connect(threaded=True)

In [6]:
if mode == "Live":
    kws.close()

In [7]:
if mode == "Live":
    print(kws.is_connected())

False
